# **Separador Flash**
Se requiere separar una mezcla de hidrógeno, acetona, alcohol isopropílico y agua en 2 corrientes de líquido y vapor a 47°C y 0.02 bar.
* *Hidrógeno*: 0.3755
* *Acetona*: 0.3755
* *Alcohol Isopropílico*: 0.0417
* *Agua*: 0.2073

La alimentación es 92.6 kmol/h

Encuentre la razón V/F y las composiciones de las corrientes de salida.


In [ ]:
import math
import numpy as np
from math import *
from tabulate import tabulate


COMP = ["Hidrogeno", "Acetona", "Alcohol Isopropilico", "Agua"]
PM=np.array([2.01589,58.08 ,60.1,18.01528])
a = np.array([2.94928, 4.21840, 8.1182, 5.11564])         # Constante A de Antoine
b = np.array([67.508, 1197.010, 1580.92, 1687.537])            # Constante B de Antoine
c = np.array([275.700, 228.060, 246.99, 219.62])              # Constante C de Antoine
z = [0.3755, 0.3755, 0.0417, 0.2073]   # Fracción mol de la Alimentación
P = 0.02                                                 # Presion [bar]
T = 47                                                  # Temperatura [°C] (320.15 K)

n = len(z)
p0 = np.zeros(n)
k = np.zeros(n)
s0 = np.zeros(n)
s1 = np.zeros(n)
for i in range (n):
    p0[i] = (10**(a[i]-(b[i]/(T+c[i]))))/750 #Cálculo de la presión de vapor por componente
    k[i] = p0[i]/P #Cálculo de la razón K (Se asume mezcla ideal)
    s0[i] = k[i]*z[i] #Cálculo de S0 y S1
    s1[i] = z[i]/k[i]

S0 = sum(s0) - 1
S1 = 1 - sum(s1)


if S0 < 0 and S1 < 0: #S0 y S1 sirven de criterio para determinar si a las condiciones propuestas existen coexistencia líquido-vapor
    print("Sólo se obtiene líquido")
if S0 > 0 and S1 > 0:
    print("Sólo se obtiene vapor")
if S0 > 0 and S1 < 0.0:
    print("Se obtiene líquido-vapor")

#Como se desconoce la relación V/F es necesario iterar
vf = 0.3 #Proponemos una relación V/F inicial
error = 1*10**(-6) #Es el error mínimo aceptable
zu = np.zeros(n)
zk = np.zeros(n)
zp = np.zeros(n)
for i in range(100):
    for i in range(n):
        zu[i] = z[i]/(1+(vf*(k[i]-1)))
        zk[i] = (z[i]*k[i])/(1+(vf*(k[i]-1)))
    sx = sum(zu)
    sy = sum(zk)
    s = sy - sx
    if abs(s) < error: #Probamos si el resultado de S es menor al error mínimo aceptable
        break
    else:
        for i in range(n): #En caso de no hacer, por medio del Método de Newton-Raphson, iteramos una nueva relación V/F
            zp[i] = (z[i]*(k[i]-1)**2)/(1+(vf)*(k[i]-1))**2
        sprim =-sum(zp)
        vf = vf - (s/sprim)


print('* La relación V/F es: %.6f' %vf)
F = 92.6 #kmol/h
V = vf*F
L = F-V
print ('* Si F = %.1f' %F, 'kmol/h')

x = np.zeros(n)
y = np.zeros(n)
for i in range (n):
    x[i] = z[i]/(1+(vf)*(k[i]-1)) #Una vez conocidad la razón V/F, se calculan las composiciones de salida en líquido y vapor de cada componente del separador flash
    y[i] = x[i]*k[i]

comp = [[' ', 'Fracción mol Vapor', 'Fracción mol Líquido'],
        ['i' ,   '   yi   ',   '   xi   '],
        [COMP[0], '%.6f.' %y[0], '%.6f.' %x[0]],
        [COMP[1], '%.6f.' %y[1], '%.6f.' %x[1]],
        [COMP[2], '%.6f.' %y[2], '%.6f.' %x[2]],
        [COMP[3], '%.6f.' %y[3], '%.6f.' %x[3]],
        ['Total', '%.6f.' %sum(y), '%.6f.' %sum(x)]]
print(tabulate(comp, headers='firstrow', tablefmt='fancy_grid'))

comp = [[' ', 'Vapor (kmol/h)', 'Líquido (kmol/h)'],
        [COMP[0], '%.6f.' %(y[0]*V), '%.6f.' %(x[0]*L)],
        [COMP[1], '%.6f.' %(y[1]*V), '%.6f.' %(x[1]*L)],
        [COMP[2], '%.6f.' %(y[2]*V), '%.6f.' %(x[2]*L)],
        [COMP[3], '%.6f.' %(y[3]*V), '%.6f.' %(x[3]*L)],
        ['Total', '%.6f.' %V, '%.6f.' %L]]
print(tabulate(comp, headers='firstrow', tablefmt='fancy_grid'))

print('\nBalance del propano: %.2f.' %((y[2]*V)+(x[2]*L)), 'kmol/h')

Se obtiene líquido-vapor
* La relación V/F es: 0.414963
* Si F = 92.6 kmol/h
╒══════════════════════╤══════════════════════╤════════════════════════╕
│                      │ Fracción mol Vapor   │ Fracción mol Líquido   │
╞══════════════════════╪══════════════════════╪════════════════════════╡
│ i                    │ yi                   │ xi                     │
├──────────────────────┼──────────────────────┼────────────────────────┤
│ Hidrogeno            │ 0.871373.            │ 0.023780.              │
├──────────────────────┼──────────────────────┼────────────────────────┤
│ Acetona              │ 0.030414.            │ 0.620268.              │
├──────────────────────┼──────────────────────┼────────────────────────┤
│ Alcohol Isopropilico │ 0.096773.            │ 0.002637.              │
├──────────────────────┼──────────────────────┼────────────────────────┤
│ Agua                 │ 0.001440.            │ 0.353315.              │
├──────────────────────┼──────────────────────┼